In [ ]:
import rust_bridge_pm_py

In [ ]:
import pm4py
import polars
import rust_bridge_pm_py
xes_path = "/home/aarkue/doc/sciebo/alpha-revisit/Road_Traffic_Fine_Management_Process.xes"

In [ ]:
# Generate huge event log (on Python side, but structs reside in Rust!)
log = rust_bridge_pm_py.native.PyBridgeEventLog()
for i in range(200000):
  trace = rust_bridge_pm_py.native.PyBridgeTrace("Trace " + str(i))
  for j in range(15):
    event = rust_bridge_pm_py.native.PyBridgeEvent({"concept:name": "Activity " + str(j)})
    trace.insert_event(j,event)
  log.insert_trace(i,trace)

In [ ]:
all_events = 0
all_events_with_same_timestamp_in_case = 0 
for g,r in log_df.groupby(["case:concept:name"]):
  events_with_same_timestamp_in_case = r.shape[0] - len(r['time:timestamp'].unique())
  all_events += r.shape[0]
  all_events_with_same_timestamp_in_case += events_with_same_timestamp_in_case
  assert events_with_same_timestamp_in_case >= 0 
  # print(events_with_same_timestamp_in_case)
print(all_events_with_same_timestamp_in_case/all_events)

In [ ]:
# Read some event log
log_df = pm4py.read_xes(xes_path)

In [ ]:
res_log = rust_bridge_pm_py.native.import_xes(xes_path).to_pandas()

In [ ]:
import pm4py
net,im,fm = pm4py.discover_petri_net_inductive(res_log)

In [ ]:
pm4py.view_petri_net(net,im,fm)

In [ ]:
import json
from rust_bridge_pm_py import petri_net
trans_net = petri_net.petrinet_to_dict(net,im,[fm])
res_net_json = rust_bridge_pm_py.native.test_petrinet(json.dumps(trans_net))
res_net =  petri_net.dict_to_petrinet(json.loads(res_net_json))
pm4py.view_petri_net(*res_net)

In [ ]:
from typing import Tuple
from pm4py.objects.petri_net.obj import PetriNet, Marking

# Checks if two Accepting Petri Nets _could_ be equal (syntactically)
# Best-effort check, does not compute full isomorphism between nets
# Partly ignores silent transitions, so use with caution
def pns_could_be_equal(a: Tuple[PetriNet,Marking,Marking], b: Tuple[PetriNet,Marking,Marking]):
  failures_reasons = []
  if len(a[0].transitions) != len(b[0].transitions):
    failures_reasons.append(f"Different number of transitions: {len(a[0].transitions)} != {len(b[0].transitions) }")
  if len(a[0].places) != len(b[0].places):
    failures_reasons.append(f"Different number of places: {len(a[0].places)} != {len(b[0].places) }")
  if len(a[0].arcs) != len(b[0].arcs):
    failures_reasons.append(f"Different number of arcs: {len(a[0].arcs)} != {len(b[0].arcs) }")
  transition_labels_a = [str(t.label) for t in a[0].transitions]
  transition_labels_b = [str(t.label) for t in b[0].transitions]
  if sorted(transition_labels_a) != sorted(transition_labels_b):
    failures_reasons.append(f"Transitions labels are not equal {list(sorted(transition_labels_a))} != {list(sorted(transition_labels_a))}")
  def extract_place_candidates(pn: PetriNet):
    candidates = {p: (set(),set()) for p in pn.places}
    for arc in pn.arcs:
      if type(arc.source) == PetriNet.Place:
        candidates[arc.source][0].add(str(arc.target.label))
      elif type(arc.source) == PetriNet.Transition:
        candidates[arc.target][1].add(str(arc.source.label))
    return {(frozenset(in_trans),frozenset(out_trans)) for (in_trans,out_trans) in candidates.values()}
  place_candidates_a = extract_place_candidates(a[0])
  place_candidates_b = extract_place_candidates(b[0])
  if place_candidates_a != place_candidates_b:
    failures_reasons.append(f"Different place candidates: {place_candidates_a} != {place_candidates_b}")
  if sorted(list(a[1].values())) != sorted(list(b[1].values())):
    failures_reasons.append(f"Initial Markings do not match: {a[1]} != {b[1]}")
  if sorted(list(a[2].values())) != sorted(list(b[2].values())):
    failures_reasons.append(f"Final Markings do not match: {a[2]} != {b[2]}")
  for reason in failures_reasons:
    print(reason)
  return len(failures_reasons) == 0


In [ ]:
pns_could_be_equal((net,im,fm),res_net)

In [ ]:
# First convert DF log to PyBridgeEventLog Wrapper and then add start/end acts.
# Result is again a PyBridgeEventLog Wrapper 
# Performance: Okay for smallish/normal logs but poor for very large ones
res_log = rust_bridge_pm_py.native.test_bridge_log(pm4py_log_to_bridge_log(log_df))

In [ ]:
# Idea: Do not require Polar dependency on python side, by exporting JSON with pandas
# and then importing it using Polar on the Rust side
log = rust_bridge_pm_py.native.test_df_pandas(log_df.to_json(orient="records"),"json")

In [ ]:
# Same as directly above, but using CSV:
log = rust_bridge_pm_py.native.test_df_pandas(log_df.to_csv(),"csv")

In [ ]:
# Idea: Do DataFrame -> Wrapper conversion on Rust side (+ in parallel)
# For that, polars is used (because of the first-class Rust support)
# First, convert log (pandas) DF to polars DF and then convert it to PyBridgeEventLog Wrapper in Rust, return Result 
# Performance: Pretty good :) 
log = rust_bridge_pm_py.native.polars_df_to_log(polars.from_pandas(log_df))

In [ ]:
# Create huge event log & process it in Rust
# For conversion, use json bytes (using orjson library)
# Not great performance...
# Total: 15308.481216430664ms; Json Dump & Log Re-construction from Dict takes the most time
# This prompted the experimentation with PyBridgeEventLog Wrapper, living in Rust
l = rust_bridge_pm_py.event_log.py_test_event_log()

In [ ]:
# Python-side PyBridgeEventLog construction: Slow!
import pandas as pd
def pm4py_log_to_bridge_log(df: pd.DataFrame):
  log = rust_bridge_pm_py.native.PyBridgeEventLog()
  for trace_id,a in df.groupby(['case:concept:name']):
    trace = rust_bridge_pm_py.native.PyBridgeTrace(str(trace_id))
    for (label,series) in a.iterrows():
      event = rust_bridge_pm_py.native.PyBridgeEvent({k: str(v) for (k,v) in series.to_dict().items()})
      trace.append_event(event)
    log.append_trace(trace)
  return log

In [ ]:
# Sample call: Add artificial start and end activities to every trace
res_log = rust_bridge_pm_py.native.test_bridge_log(log)
assert res_log.traces[0].events[0].attributes.get("concept:name") == "__START__"
assert res_log.traces[0].events[-1].attributes.get("concept:name") == "__END__"

In [ ]:
res['arcs'][0]

In [ ]:
import pm4py
from pm4py.objects.petri_net.obj import PetriNet


def petrinet_to_dict(net: PetriNet) -> dict:
    places = {p.name: {"id": p.name} for p in net.places}
    transitions = {t.name: {"id": t.name, "label": t.label} for t in net.transitions}
    arcs = [
        {
            "from_to": {
                "type": "PlaceTransition"
                if type(arc.source) == PetriNet.Place
                else "TransitionPlace",
                "nodes": [arc.source.name, arc.target.name],
            },
            "weight": arc.weight,
        }
        for arc in net.arcs
    ]
    return {"places": places, "transitions": transitions, "arcs": arcs}

In [ ]:
petrinet_to_dict(dict_to_petrinet(res)) == res

In [ ]:
import pm4py
from pm4py.objects.petri_net.obj import PetriNet


def dict_to_petrinet(net_dict) -> PetriNet:
    places = {p["id"]: PetriNet.Place(p["id"]) for p in net_dict["places"].values()}
    transitions = {
        t["id"]: PetriNet.Transition(t["id"], t["label"])
        for t in net_dict["transitions"].values()
    }

    def get_arc_for(arc_dict):
        if arc_dict["from_to"]["type"] == "PlaceTransition":
            fr = places.get(arc_dict["from_to"]["nodes"][0])
            to = transitions.get(arc_dict["from_to"]["nodes"][1])
        else:
            fr = transitions.get(arc_dict["from_to"]["nodes"][0])
            to = places.get(arc_dict["from_to"]["nodes"][1])
        return PetriNet.Arc(fr, to, arc_dict["weight"])

    arcs = [get_arc_for(arc_dict) for arc_dict in net_dict["arcs"]]
    net = PetriNet(None, places.values(), transitions.values(), arcs)
    return net

In [ ]:
pm4py.view_petri_net(dict_to_petrinet(res))

In [ ]:
import rust_bridge_pm_py
import json
res = json.loads(rust_bridge_pm_py.native.test_petrinet())
res

In [ ]:
# Transform PyBridgeEventLog to dict (only keeping activity names + case id)
traces = []
for trace in log.traces:
  events = []
  for event in trace.events:
    events.append({"concept:name": event.attributes.get("concept:name")})
  traces.append({"case:concept:name": trace.attributes.get("case:concept:name"),"events": events})
